In [ ]:
import os
print(os.listdir('../input/polyline-curves'))

import numpy as np

# Read Polyline Data
def read_csv(csv_path):
    np_path_XYs = np.genfromtxt(csv_path, delimiter=',')
    path_XYs = []
    for i in np.unique(np_path_XYs[:, 0]):
        npXYs = np_path_XYs[np_path_XYs[:, 0] == i][:, 1:]
        XYs = []
        for j in np.unique(npXYs[:, 0]):
            XY = npXYs[npXYs[:, 0] == j][:, 1:]
            XYs.append(XY)
        path_XYs.append(XYs)
    return path_XYs

# Visualize the Polyline Data
import matplotlib.pyplot as plt

def plot(paths_XYs, colours=['r', 'g', 'b', 'c', 'm', 'y', 'k']):
    fig, ax = plt.subplots(tight_layout=True, figsize=(8, 8))
    for i, XYs in enumerate(paths_XYs):
        c = colours[i % len(colours)]
        for XY in XYs:
            ax.plot(XY[:, 0], XY[:, 1], c=c, linewidth=2)
    ax.set_aspect('equal')
    plt.show()

# Example usage
paths_XYs = read_csv('../input/polyline-curves/frag0.csv')
plot(paths_XYs)

paths_XYs = read_csv('../input/polyline-curves/frag01_sol.csv')
plot(paths_XYs)

paths_XYs = read_csv('../input/polyline-curves/frag1.csv')
plot(paths_XYs)

paths_XYs = read_csv('../input/polyline-curves/frag2.csv')
plot(paths_XYs)

paths_XYs = read_csv('../input/polyline-curves/frag2_sol.csv')
plot(paths_XYs)

paths_XYs = read_csv('../input/polyline-curves/isolated.csv')
plot(paths_XYs)

paths_XYs = read_csv('../input/polyline-curves/isolated_sol.csv')
plot(paths_XYs)

paths_XYs = read_csv('../input/polyline-curves/occlusion1.csv')
plot(paths_XYs)

paths_XYs = read_csv('../input/polyline-curves/occlusion1_sol.csv')
plot(paths_XYs)

paths_XYs = read_csv('../input/polyline-curves/occlusion2_sol.csv')
plot(paths_XYs)


# Identifying and Regularizing curves
def regularize_curves(paths_XYs):
    regular_shapes = []
    for path in paths_XYs:
        for polyline in path:
            # Identify straight lines
            if is_straight_line(polyline):
                regular_shapes.append(('line', polyline))
            # Identify circles
            elif is_circle(polyline):
                regular_shapes.append(('circle', polyline))
            # Identify rectangles and rounded rectangles
            elif is_rectangle(polyline):
                regular_shapes.append(('rectangle', polyline))
            elif is_rounded_rectangle(polyline):
                regular_shapes.append(('rounded_rectangle', polyline))
            # Identify regular polygons
            elif is_regular_polygon(polyline):
                regular_shapes.append(('polygon', polyline))
            # Identify star shapes
            elif is_star_shape(polyline):
                regular_shapes.append(('star', polyline))
            else:
                regular_shapes.append(('unknown', polyline))
    return regular_shapes

import numpy as np

def is_straight_line(polyline):
    # Ensure there are at least two points to form a line
    if len(polyline) < 2:
        return False

    # Calculate the slope between the first two points
    x0, y0 = polyline[0]
    x1, y1 = polyline[1]

    if x1 == x0:
        initial_slope = float('inf')  # Vertical line
    else:
        initial_slope = (y1 - y0) / (x1 - x0)

    # Check the slope for all subsequent pairs
    for i in range(1, len(polyline) - 1):
        x0, y0 = polyline[i]
        x1, y1 = polyline[i + 1]

        if x1 == x0:
            slope = float('inf')
        else:
            slope = (y1 - y0) / (x1 - x0)

        if slope != initial_slope:
            return False

    return True


def is_circle(polyline, tolerance=1e-10):
    # Ensure there are at least three points to form a circle
    if len(polyline) < 3:
        return False

    # Calculate the centroid of the points
    centroid = np.mean(polyline, axis=0)

    # Calculate the distances from the centroid to each point
    distances = np.linalg.norm(polyline - centroid, axis=1)

    # Check if all distances are approximately equal
    mean_distance = np.mean(distances)
    if np.all(np.abs(distances - mean_distance) < tolerance):
        return True
    else:
        return False


def is_rectangle(polyline, tolerance=1e-10):
    # Ensure there are exactly four points
    if len(polyline) != 4:
        return False

    def distance(p1, p2):
        return np.linalg.norm(p1 - p2)

    def angle(p1, p2, p3):
        v1 = p1 - p2
        v2 = p3 - p2
        cos_theta = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
        return np.arccos(cos_theta)

    # Calculate distances between consecutive points
    d = [distance(polyline[i], polyline[(i + 1) % 4]) for i in range(4)]

    # Check if opposite sides are equal
    if not (np.abs(d[0] - d[2]) < tolerance and np.abs(d[1] - d[3]) < tolerance):
        return False

    # Check if angles are 90 degrees
    for i in range(4):
        theta = angle(polyline[i], polyline[(i + 1) % 4], polyline[(i + 2) % 4])
        if not np.abs(theta - np.pi / 2) < tolerance:
            return False

    return True


def is_rounded_rectangle(polyline, tolerance=1e-10):
     # Ensure there are at least 8 points to represent a rounded rectangle
    if len(polyline) < 8:
        return False

    def distance(p1, p2):
        return np.linalg.norm(p1 - p2)

    def angle(p1, p2, p3):
        v1 = p1 - p2
        v2 = p3 - p2
        cos_theta = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
        return np.arccos(cos_theta)

    def is_straight_segment(points):
        if len(points) < 2:
            return False
        p0 = points[0]
        p1 = points[-1]
        for p in points[1:-1]:
            if not np.abs(np.cross(p1 - p0, p - p0)) < tolerance:
                return False
        return True

    def is_rounded_corner(points):
        if len(points) < 4:
            return False
        center = np.mean(points, axis=0)
        distances = np.linalg.norm(points - center, axis=1)
        mean_distance = np.mean(distances)
        return np.all(np.abs(distances - mean_distance) < tolerance)

    n = len(polyline)
    segment_length = (n-4) // 4

    # Ensure the polyline is closed
    if not np.allclose(polyline[0], polyline[-1], atol=tolerance):
        return False


    straight_segments = [
        polyline[i*segment_length : (i+1)*segment_length] for i in range(4)
    ]

    rounded_corners = [
        polyline[(i+1)*segment_length : (i+2)*segment_length] for i in range(4)
    ]

    for segment in straight_segments:
        if not is_straight_segment(segment):
            return False

    for corner in rounded_corners:
        if not is_rounded_corner(corner):
            return False

    return True

#print(is_rounded_rectangle(polyline))  # Output should be False

def is_regular_polygon(polyline, tolerance=1e-5):
     # Ensure there are at least three points to form a polygon
    if len(polyline) < 3:
        return False

    def distance(p1, p2):
        return np.linalg.norm(p1 - p2)

    def angle(p1, p2, p3):
        v1 = p1 - p2
        v2 = p3 - p2
        cos_theta = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
        return np.arccos(np.clip(cos_theta, -1.0, 1.0))

    n = len(polyline)

    # Check if the number of points is consistent with a regular polygon
    if n < 3 or n % 2 != 0:
        return False

    # Calculate distances between consecutive points
    distances = [distance(polyline[i], polyline[(i + 1) % n]) for i in range(n)]

    # Check if all sides are equal
    mean_distance = np.mean(distances)
    if not np.all(np.abs(distances - mean_distance) < tolerance):
        return False

    # Calculate angles between consecutive sides
    angles = [angle(polyline[i], polyline[(i + 1) % n], polyline[(i + 2) % n]) for i in range(n)]

    # Check if all angles are equal
    mean_angle = np.mean(angles)
    if not np.all(np.abs(angles - mean_angle) < tolerance):
        return False

    return True


def is_star_shape(polyline, tolerance=1e-5):
    # Ensure there are enough points to form a star shape
    if len(polyline) < 10:
        return False

    def distance(p1, p2):
        return np.linalg.norm(p1 - p2)

    def angle(p1, p2, p3):
        v1 = p1 - p2
        v2 = p3 - p2
        cos_theta = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
        return np.arccos(np.clip(cos_theta, -1.0, 1.0))

    def check_star_pattern(points):
        n = len(points)
        if n % 2 != 0:
            return False

        # Divide points into outer and inner sets
        outer_points = points[0::2]
        inner_points = points[1::2]

        if len(outer_points) != len(inner_points):
            return False

        # Check distances between consecutive outer and inner points
        outer_distances = [distance(outer_points[i], outer_points[(i + 1) % len(outer_points)]) for i in range(len(outer_points))]
        inner_distances = [distance(inner_points[i], inner_points[(i + 1) % len(inner_points)]) for i in range(len(inner_points))]

        mean_outer_distance = np.mean(outer_distances)
        mean_inner_distance = np.mean(inner_distances)

        if not (np.all(np.abs(outer_distances - mean_outer_distance) < tolerance) and
                np.all(np.abs(inner_distances - mean_inner_distance) < tolerance)):
            return False

        # Check angles at each vertex
        for i in range(len(outer_points)):
            p1 = outer_points[i]
            p2 = outer_points[(i + 1) % len(outer_points)]
            p3 = inner_points[i]
            p4 = inner_points[(i + 1) % len(inner_points)]

            angle1 = angle(p1, p2, p3)
            angle2 = angle(p3, p4, p1)

            if not (np.abs(angle1 - 2* np.pi / 5) < tolerance and np.abs(angle2 - 2 * np.pi / 5) < tolerance):
                return False

        return True

    return check_star_pattern(polyline)

# Symmetry
def find_symmetries(paths_XYs):
    symmetries = []
    for path in paths_XYs:
        for polyline in path:
            if has_reflection_symmetry(polyline):
                symmetries.append(('reflection', polyline))
            elif has_rotational_symmetry(polyline):
                symmetries.append(('rotation', polyline))
            else:
                symmetries.append(('none', polyline))
    return symmetries

def has_reflection_symmetry(polyline, tolerance=1e-5):
    def distance(p1, p2):
        return np.linalg.norm(p1 - p2)

    def reflect_point(p, axis):
        """ Reflect point p over a line defined by axis (a, b, c). """
        a, b, c = axis
        x0, y0 = p
        x = (x0 - 2 * a * (a * x0 + b * y0 + c) / (a ** 2 + b ** 2))
        y = (y0 - 2 * b * (a * x0 + b * y0 + c) / (a ** 2 + b ** 2))
        return np.array([x, y])

    def line_from_points(p1, p2):
        """ Compute line coefficients (a, b, c) from two points (p1, p2). """
        x1, y1 = p1
        x2, y2 = p2
        a = y2 - y1
        b = x1 - x2
        c = x2 * y1 - x1 * y2
        return a, b, c

    def all_points_symmetric(points, axis):
        """ Check if all points are symmetric with respect to the given axis. """
        reflected_points = [reflect_point(p, axis) for p in points]
        reflected_points = np.array(reflected_points)
        points = np.array(points)
        # Check if each point has a corresponding reflected point within tolerance
        for p in points:
            if not any(np.linalg.norm(p - rp) < tolerance for rp in reflected_points):
                return False
        return True

    n = len(polyline)

    if n < 2:
        return False

    # Check symmetry for axes defined by pairs of points
    for i in range(n):
        for j in range(i + 1, n):
            p1 = polyline[i]
            p2 = polyline[j]
            axis = line_from_points(p1, p2)
            if all_points_symmetric(polyline, axis):
                return True

    return False



def has_rotational_symmetry(polyline, tolerance=1e-5):
    def rotate_point(p, angle, center):
        """ Rotate point p around center by angle radians. """
        cos_angle = np.cos(angle)
        sin_angle = np.sin(angle)
        x, y = p
        cx, cy = center
        x_new = cos_angle * (x - cx) - sin_angle * (y - cy) + cx
        y_new = sin_angle * (x - cx) + cos_angle * (y - cy) + cy
        return np.array([x_new, y_new])

    def all_points_symmetric(points, angle, center):
        """ Check if rotating all points by angle around center maps them onto the original set. """
        rotated_points = [rotate_point(p, angle, center) for p in points]
        rotated_points = np.array(rotated_points)
        points = np.array(points)
        # Check if each point has a corresponding rotated point within tolerance
        for p in points:
            if not any(np.linalg.norm(p - rp) < tolerance for rp in rotated_points):
                return False
        return True

    def calculate_center(points):
        """ Calculate the centroid of the polygon. """
        return np.mean(points, axis=0)

    n = len(polyline)
    if n < 3:
        return False

    center = calculate_center(polyline)

    # Check for rotational symmetry for different angles
    for k in range(1, n):
        angle = 2 * np.pi * k / n
        if all_points_symmetric(polyline, angle, center):
            return True

    return False
# Complete Incomplete curve
def complete_curves(paths_XYs):
    completed_paths = []
    for path in paths_XYs:
        for polyline in path:
            if is_incomplete(polyline):
                completed_polyline = complete_polyline(polyline)
                completed_paths.append(completed_polyline)
            else:
                completed_paths.append(polyline)
    return completed_paths

def is_incomplete(polyline, threshold=0.05):
#"""
   # Determine if a polyline is incomplete based on its endpoints.

   # Parameters:
    #    polyline (list of tuples): List of (x, y) tuples representing the polyline.
    #    threshold (float): The distance threshold to determine if the polyline is incomplete.

   # Returns:
   #     bool: True if the polyline is incomplete, False otherwise.
   # """
    if len(polyline) < 2:
        # A polyline with less than 2 points can't be incomplete
        return False

    start_point = np.array(polyline[0])
    end_point = np.array(polyline[-1])

    # Calculate the Euclidean distance between the start and end points
    distance = np.linalg.norm(start_point - end_point)

    # If the distance is greater than the threshold, consider it incomplete
    return distance > threshold




def complete_polyline(polyline):
    from scipy.interpolate import CubicSpline

def complete_polyline(polyline):
    # Identify gaps in the polyline
    gaps = identify_gaps(polyline)

    completed_polyline = list(polyline)  # Copy the original polyline

    for gap in gaps:
        start, end = gap  # Define start and end points of the gap
        # Fit a curve to the points surrounding the gap
        curve = fit_curve(completed_polyline[start-1:end+1])
        # Generate points to fill the gap
        missing_points = generate_points(curve, start, end)
        # Insert the missing points into the polyline
        completed_polyline[start:end] = missing_points

    return completed_polyline

def identify_gaps(polyline):
    # Dummy function to find gaps
    gaps = []
    for i in range(1, len(polyline)):
        if np.linalg.norm(np.array(polyline[i]) - np.array(polyline[i-1])) > threshold:
            gaps.append((i-1, i))
    return gaps

def fit_curve(points):
    # Example using cubic spline for curve fitting
    x = [p[0] for p in points]
    y = [p[1] for p in points]
    return CubicSpline(x, y)

def generate_points(curve, start, end):
    # Generate points along the curve
    x_new = np.linspace(start, end, num=10)
    y_new = curve(x_new)
    return list(zip(x_new, y_new))

!pip install svgwrite cairosvg

 # Convert Polylines to SVG and Rasterize
import svgwrite
import cairosvg

def polylines2svg(paths_XYs, svg_path, colours=['red', 'green', 'blue']):
    W, H = 0, 0
    for path_XYs in paths_XYs:
        for XY in path_XYs:
            W, H = max(W, np.max(XY[:, 0])), max(H, np.max(XY[:, 1]))
    padding = 0.1
    W, H = int(W + padding * W), int(H + padding * H)

    dwg = svgwrite.Drawing(svg_path, profile='tiny', shape_rendering='crispEdges')
    group = dwg.g()
    for i, path in enumerate(paths_XYs):
        path_data = []
        c = colours[i % len(colours)]
        for XY in path:
            path_data.append(("M", (XY[0, 0], XY[0, 1])))
            for j in range(1, len(XY)):
                path_data.append(("L", (XY[j, 0], XY[j, 1])))
            if not np.allclose(XY[0], XY[-1]):
                path_data.append(("Z", None))
        group.add(dwg.path(d=path_data, fill=c, stroke='none', stroke_width=2))
    dwg.add(group)
    dwg.save()

    png_path = svg_path.replace('.svg', '.png')
    fact = max(1, 1024 // min(H, W))
    cairosvg.svg2png(url=svg_path, write_to=png_path, parent_width=W, parent_height=H, output_width=fact * W, output_height=fact * H, background_color='white')

# Example usage
polylines2svg(paths_XYs, 'output.svg')



# Evaluation Module
def evaluate_shapes(detected_shapes, expected_shapes):
    correct = 0
    for detected, expected in zip(detected_shapes, expected_shapes):
        if detected[0] == expected[0]:
            correct += 1
    accuracy = correct / len(expected_shapes) if expected_shapes else 0
    print(f'Shape Detection Accuracy: {accuracy:.2f}')
    return accuracy

def evaluate_symmetry(detected_symmetry, expected_symmetry):
    correct = 0
    for detected, expected in zip(detected_symmetry, expected_symmetry):
        if detected[0] == expected[0]:
            correct += 1
    accuracy = correct / len(expected_symmetry) if expected_symmetry else 0
    print(f'Symmetry Detection Accuracy: {accuracy:.2f}')
    return accuracy

def evaluate_completion(detected_paths_XYs, expected_paths_XYs):
    correct = 0
    for detected, expected in zip(detected_paths_XYs, expected_paths_XYs):
        if np.allclose(np.array(detected), np.array(expected)):
            correct += 1
    accuracy = correct / len(expected_paths_XYs) if expected_paths_XYs else 0
    print(f'Curve Completion Accuracy: {accuracy:.2f}')
    return accuracy

# Example Usage of the Evaluation Module
# Assuming we have expected_shapes, expected_symmetry, and expected_paths_XYs

detected_shapes = regularize_curves(paths_XYs)
detected_symmetry = find_symmetries(paths_XYs)
detected_completion = complete_curves(paths_XYs)

# Placeholder for expected results
expected_shapes = detected_shapes  # Normally this would be your ground truth data
expected_symmetry = detected_symmetry  # Ground truth data
expected_completion = detected_completion  # Ground truth data

evaluate_shapes(detected_shapes, expected_shapes)
evaluate_symmetry(detected_symmetry, expected_symmetry)
evaluate_completion(detected_completion, expected_completion)